<a href="https://colab.research.google.com/github/dchlseo/DataScienceProjects/blob/main/DeepLearningBasics/TensorFlow/10_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tensorboard
- https://www.tensorflow.org/tensorboard?hl=ko
- [tensorboard in google colab](https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(7777)
tf.random.set_seed(7777)

In [3]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])

        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(16, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)

    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)

    net1_1 = Conv2D(32, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])

    net1 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(32, kernel_size=1, padding='same', activation='relu')(net2)

    net = Add()([net, net3])

    net = MaxPool2D()(net)

    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')

    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 16)           448       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 8, 8, 16)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 8, 8, 32)             544       ['max_pooling2d[0][0]']  

In [12]:
learning_rate = 0.03
opt = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.binary_crossentropy

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

### Logs using callbacks

In [21]:
import datetime
log_dir = os.path.join("logs/fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [8]:
log_dir

'logs/fit20231106-050455'

In [10]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [13]:
model.fit(x=train_x,
          y=train_y,
          epochs=5,
          validation_data=(test_x, test_y),
          callbacks=[tb_callback]
          )

Epoch 1/5
1563/1563 [==============================] - 34s 20ms/step - loss: 0.2782 - accuracy: 0.3422 - val_loss: 0.2765 - val_accuracy: 0.3340
Epoch 2/5
1563/1563 [==============================] - 32s 21ms/step - loss: 0.2702 - accuracy: 0.3661 - val_loss: 0.2638 - val_accuracy: 0.4055
Epoch 3/5
1563/1563 [==============================] - 31s 20ms/step - loss: 0.2706 - accuracy: 0.3657 - val_loss: 0.2753 - val_accuracy: 0.3581
Epoch 4/5
1563/1563 [==============================] - 34s 22ms/step - loss: 0.2698 - accuracy: 0.3702 - val_loss: 0.2624 - val_accuracy: 0.3964
Epoch 5/5
1563/1563 [==============================] - 29s 19ms/step - loss: 0.2700 - accuracy: 0.3735 - val_loss: 0.3019 - val_accuracy: 0.3175


In [17]:
%load_ext tensorboard
import os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
!tensorboard --logdir log_dir

2023-11-06 05:16:55.964854: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 05:16:55.964930: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 05:16:55.964968: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 05:16:57.243571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Address already in use
Port 6006 is in use b

In [24]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir log_dir (started 0:00:34 ago; pid 4228)


### using tf.summary

In [26]:
loss_fn = tf.keras.losses.categorical_crossentropy

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x, y) :
    with tf.GradientTape() as tape:
        pred = model(x)
        loss = loss_fn(y, pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y, pred)

@tf.function
def test_step(x, y) :

    pred = model(x)
    loss = loss_fn(y, pred)

    test_loss(loss)
    test_accuracy(y, pred)

In [27]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'

In [28]:
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [29]:
batch_size = 64

num_of_batch_train = train_x.shape[0] // batch_size
num_of_batch_test = test_x.shape[0] // batch_size

for epoch in range(5):

    for i in range(num_of_batch_train):
        idx = i * batch_size
        x, y = train_x[idx:idx+batch_size], train_y[idx:idx+batch_size]
        train_step(x, y)
        print("\r Train : {} / {}".format(i, num_of_batch_train), end='\r')


    for i in range(num_of_batch_test):
        idx = i * batch_size
        x, y = test_x[idx:idx+batch_size], test_y[idx:idx+batch_size]
        test_step(x, y)
        print("\r Test : {} / {}".format(i, num_of_batch_test), end='\r')

    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('acc', train_accuracy.result(), step=epoch)

    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('acc', test_accuracy.result(), step=epoch)

    fmt = 'epoch {} loss: {}, accuracy: {}, test_loss: {}, test_acc: {}'
    print(fmt.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result(),
                          test_loss.result(),
                          test_accuracy.result()
                    )
         )

    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

epoch 1 loss: 2.7219741344451904, accuracy: 0.09879161417484283, test_loss: 2.3220443725585938, test_acc: 0.10006009787321091
epoch 2 loss: 2.314493179321289, accuracy: 0.0981714129447937, test_loss: 2.3142189979553223, test_acc: 0.10006009787321091
epoch 3 loss: 2.310041666030884, accuracy: 0.09805137664079666, test_loss: 2.3195512294769287, test_acc: 0.10006009787321091
epoch 4 loss: 2.3103299140930176, accuracy: 0.09893165528774261, test_loss: 2.32029390335083, test_acc: 0.09985977411270142
epoch 5 loss: 2.3151516914367676, accuracy: 0.10065221041440964, test_loss: 2.32436466217041, test_acc: 0.09985977411270142


In [31]:
!tensorboard --logdir logs/gradient_tape --load_fast=false

2023-11-06 05:20:41.657048: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 05:20:41.657111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 05:20:41.657147: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 05:20:42.845825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [32]:
logdir = "logs/train_data/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

for i in np.random.randint(10000, size=10):
    img = train_x[i:i+1]
    with file_writer.as_default():
        tf.summary.image("Training Sample data : {}".format(i), img, step=0)

In [ ]:
!tensorboard --logdir logs/train_data

### LambdaCallback을 사용하여 Tensorboard에 Confusion Matrix 기록

In [ ]:
import io
from sklearn.metrics import confusion_matrix

def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

def plot_confusion_matrix(cm, class_names):

    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    threshold = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

logdir = "logs/fit/cm/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer_cm = tf.summary.create_file_writer(logdir)

test_images = test_x[:100]
test_labels = np.argmax(test_y[:100], axis=1)

def log_confusion_matrix(epoch, logs):
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    classes = np.arange(10)
    cm = confusion_matrix(test_labels, test_pred, labels=classes)

    figure = plot_confusion_matrix(cm, class_names=classes)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [33]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

NameError: ignored

In [ ]:
model.fit(x=train_x,
          y=train_y,
          epochs=5,
          batch_size=32,
          validation_data=(test_x, test_y),
          callbacks=[tensorboard_callback, cm_callback])

In [ ]:
!tensorboard --logdir logs/fit